In [1]:
from pathlib import Path

from ase import units, Atoms
from ase.build import molecule
from ase.io import read, write
from dask.distributed import Client
from dask_jobqueue import SLURMCluster
from prefect import flow
from prefect_dask import DaskTaskRunner
from pymatgen.core import Molecule
from pymatgen.io.packmol import PackmolBoxGen

from mlip_arena.models.utils import REGISTRY, MLIPEnum
from mlip_arena.tasks import MD

In [ ]:
h2 = molecule("H2")
o2 = molecule("O2")
h2o = molecule("H2O")

write("h2.xyz", h2)
write("o2.xyz", o2)
write("h2o.xyz", h2o)

In [ ]:
h2 = Molecule.from_file("h2.xyz")
o2 = Molecule.from_file("o2.xyz")
h2o = Molecule.from_file("h2o.xyz")

In [ ]:
molecules = []

for m, number in zip([h2, o2], [128, 64]):
    molecules.append(
        {
            "name": m.composition.to_pretty_string(),
            "number": number,
            "coords": m,
        }
    )

In [ ]:
tolerance = 2.0
input_gen = PackmolBoxGen(
    tolerance=tolerance,
    seed=1,
)
margin = 0.5 * tolerance

a = 30

packmol_set = input_gen.get_input_set(
    molecules=molecules,
    box=[margin, margin, margin, a - margin, a - margin, a - margin],
)
packmol_set.write_input(".")
packmol_set.run(".")

atoms = read("packmol_out.xyz")
atoms.cell = [a, a, a]
atoms.pbc = True

print(atoms)

write(f'{atoms.get_chemical_formula()}.extxyz', atoms)

In [2]:
atoms = read("H256O128.extxyz")
print(atoms)

Atoms(symbols='H256O128', pbc=True, cell=[30.0, 30.0, 30.0])


In [3]:
nodes_per_alloc = 1
gpus_per_alloc = 4
ntasks = 1

cluster_kwargs = dict(
    cores=1,
    memory="64 GB",
    shebang="#!/bin/bash",
    account="matgen",
    walltime="02:00:00",
    job_mem="0",
    job_script_prologue=[
        "source ~/.bashrc",
        "module load python",
        "source activate /pscratch/sd/c/cyrusyc/.conda/mlip-arena",
    ],
    job_directives_skip=["-n", "--cpus-per-task", "-J"],
    job_extra_directives=[
        "-J combustion-water",
        "-q regular",
        f"-N {nodes_per_alloc}",
        "-C gpu",
        f"-G {gpus_per_alloc}",
        # f"--exclusive",
        # "--time-min=00:30:00",
        # "--comment=1-00:00:00",
        # "--signal=B:USR1@60",
        # "--requeue",
        # "--open-mode=append"
    ],
    death_timeout=86400
)


cluster = SLURMCluster(**cluster_kwargs)
print(cluster.job_script())
cluster.adapt(minimum_jobs=5, maximum_jobs=50)
client = Client(cluster)

#!/bin/bash

#SBATCH -A matgen
#SBATCH --mem=0
#SBATCH -t 02:00:00
#SBATCH -J combustion-water
#SBATCH -q regular
#SBATCH -N 1
#SBATCH -C gpu
#SBATCH -G 4
source ~/.bashrc
module load python
source activate /pscratch/sd/c/cyrusyc/.conda/mlip-arena
/pscratch/sd/c/cyrusyc/.conda/mlip-arena/bin/python -m distributed.cli.dask_worker tcp://128.55.64.32:35601 --name dummy-name --nthreads 1 --memory-limit 59.60GiB --nanny --death-timeout 86400



In [4]:
@flow(task_runner=DaskTaskRunner(address=client.scheduler.address), log_prints=True)
def combustion(atoms: Atoms):
    futures = []

    for model in MLIPEnum:
        future = MD.submit(
            atoms=atoms,
            calculator_name=model,
            calculator_kwargs=None,
            ensemble="nvt",
            dynamics="nose-hoover",
            time_step=None,
            ase_md_kwargs=dict(ttime=25 * units.fs, pfactor=None),
            total_time=1000_000,
            temperature=[300, 3000, 3000, 300],
            pressure=None,
            mb_velocity_seed=0,
            traj_file=Path(REGISTRY[model.name]["family"])
            / f"{atoms.get_chemical_formula()}.traj",
            traj_interval=1000,
            restart=True,
        )

        futures.append(future)
    
    return [future.result() for future in futures]

In [ ]:
results = combustion(atoms)

10:53:24.417 | INFO    | prefect.engine - Created flow run 'masterful-panther' for flow 'combustion'

10:53:24.422 | INFO    | Flow run 'masterful-panther' - View at https://app.prefect.cloud/account/f7d40474-9362-4bfa-8950-ee6a43ec00f3/workspace/d4bb0913-5f5e-49f7-bfc5-06509088baeb/flow-runs/flow-run/4d0a29f8-4313-4dd3-b311-ea64abf82a83

10:53:24.423 | INFO    | prefect.task_runner.dask - Connecting to an existing Dask cluster at tcp://128.55.64.32:35601

10:53:24.430 | INFO    | prefect.task_runner.dask - The Dask dashboard is available at http://128.55.64.32:8787/status

10:53:24.927 | INFO    | Flow run 'masterful-panther' - Created task run 'md-4' for task 'md'

...

10:53:25.623 | INFO    | Flow run 'masterful-panther' - Submitted task run 'md-2' for execution.

10:53:25.638 | INFO    | Flow run 'masterful-panther' - Submitted task run 'md-4' for execution.

10:53:25.713 | INFO    | Flow run 'masterful-panther' - Submitted task run 'md-5' for execution.

10:53:25.831 | INFO    | Flow run 'masterful-panther' - Submitted task run 'md-6' for execution.

10:53:25.903 | INFO    | Flow run 'masterful-panther' - Submitted task run 'md-7' for execution.